In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab

from collections import Counter

from IPython.core.display import HTML, display
from models.generators import DiffusionModel
from models.discriminators import DiscriminatorNetCIFAR10
from torchvision import transforms
import torch.nn.functional as F
from captum.attr import visualization as viz
from captum.attr import Lime, LimeBase
from captum._utils.models.linear_model import SkLearnLinearRegression, SkLearnLasso
import os
import json

In [5]:
class model():
    def __init__(self,pretrained_path):
        
        self.encoder = DiffusionModel()
        #self.decoder = DiscriminatorNetCIFAR10()
        self.decoder = DiscriminatorNetCIFAR10()
        #discriminator = discriminator.load_state_dict(torch.load(pretrained_path, map_location=lambda storage, loc: storage)['model_state_dict'])
        self.decoder.load_state_dict(torch.load(pretrained_path, map_location=lambda storage, loc: storage)['model_state_dict'])
        self.transform = transforms.Compose([
            transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),transforms.Resize((32,32))])
    
    def forward(self, x):
        label = self.encoder.forward(x)
        
        label = self.transform(label)
        label = label.unsqueeze(0)
        #print(x.shape)
        
        self.decoder.eval()
        label = self.decoder(label)
        return label
    
    def encode_part(self,input):
        return self.encoder.forward(input)

In [6]:
pretrained_path = r"D:\Praktikum\xai-diffusion\xai-praktikum\xaigan\src\results\cifar-10\CIFAR10_only_SaliencyTrain\discriminator.pt"
model = model(pretrained_path)

text_inputs = ["A glass of beer is sitting next to a vase full of flowers."]
label = model.forward(text_inputs)
print(label)
print(label.shape)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
d:\Anaconda\envs\mlgs\lib\site-packages\transformers\models\clip\feature_extraction_clip.py:31: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  FutureWarning,


  0%|          | 0/50 [00:00<?, ?it/s]

tensor([[[[0.9435]]]], grad_fn=<SigmoidBackward0>)
torch.Size([1, 1, 1, 1])


In [7]:
def predict(text):
    # stack up all images
    prob = model.forward(text).squeeze()
    return prob.unsqueeze(0).unsqueeze(0).detach().numpy()
text = "A glass of beer is sitting next to a vase full of flowers."
#a = predict(text)
#print(a.shape)
#print(a)

In [8]:
from lime import lime_text
from lime.lime_text import LimeTextExplainer

class_names = ["false"]
text = "A glass of beer is sitting next to a vase full of flowers."
explainer = LimeTextExplainer(class_names=class_names)
exp = explainer.explain_instance(text, predict,num_features = 2,num_samples=3, labels=(0,))
#print('Document id: %d' % idx)
#print(text)
#print('Predicted class =', class_names[model.forward(text).reshape(1,-1)[0,0]])
#print('True class: %s' % true_label)
exp.as_list()
exp.show_in_notebook(text=text, labels=(1,))

  0%|          | 0/50 [00:00<?, ?it/s]

(3, 1)
(1,)


ValueError: Found input variables with inconsistent numbers of samples: [3, 1]